### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

In [ ]:
import numpy as np
import pandas as pd
import json
import codecs
import os
import sys
import datetime as dt
sys.path.append('../')
import oci
import ocifs

In [ ]:
from generate_data_extraction.features_functions import *
from generate_data_extraction.reduce_feature_list import *
from generate_data_extraction.common_columns_lists import *

In [ ]:
bucket_name = "tenen-raw-data"
config = oci.config.from_file(
        "~/.apikey/config",
        "DEFAULT")
object_storage_client = oci.object_storage.ObjectStorageClient(config)
namespace = object_storage_client.get_namespace().data
namespace

In [ ]:
hl_file = {'oct' : 'tenen_high_lvl_oct.csv', 'nov': 'tenen_high_lvl_nov.csv', 'dec': 'tenen_high_lvl_dec.csv' }
hits_file = {'oct' : 'tenen_hits_extend_oct.csv', 'nov': 'tenen_hits_extend_nov.csv', 'dec': 'tenen_hits_extend_dec.csv'}

In [ ]:
fname = hl_file['dec']
directory = 'products'
config = {"config": "~/.apikey/config"}
target = f'oci://{bucket_name}@{namespace}/{directory}/{fname}'
target

In [ ]:
X1 = pd.read_csv(target, storage_options={"config": "~/.apikey/config"},low_memory=False)
X1.shape

In [ ]:
#namespace = "lrhwvlagph9z"
df_tl_oct = pd.read_csv(target, storage_options={"config": "~/.apikey/config"},low_memory=False)
df_tl_oct.shape

In [ ]:
df_tl_nov = pd.read_csv(target, storage_options={"config": "~/.apikey/config"},low_memory=False)
df_tl_nov.shape

In [ ]:
df_tl_dec = pd.read_csv(target, storage_options={"config": "~/.apikey/config"},low_memory=False)
df_tl_dec.shape

In [ ]:
df_tl_oct['fullVisitorId'] = df_tl_oct['fullVisitorId'].astype(str)
df_tl_nov['fullVisitorId'] = df_tl_nov['fullVisitorId'].astype(str)
df_tl_dec['fullVisitorId'] = df_tl_dec['fullVisitorId'].astype(str)

In [ ]:
df_list = [df_tl_oct, df_tl_nov, df_tl_dec]

In [ ]:
df = pd.concat(df_list, ignore_index=True)
df.shape

In [ ]:
numeric_cols = ['totals.hits', 'totals.pageviews', 'totals.timeOnSite']

In [ ]:
df_tl = init_top_lvl_df(df, numeric_cols)
df_tl.shape

In [ ]:
tmp = df_tl.groupby('fullVisitorId')['fullVisitorId'].agg('count')
len(tmp)

In [ ]:
X = emotional_rational_toplvl_features(df_tl)
X.shape

In [ ]:
X.isnull().values.any()

In [ ]:
X.head(5)

In [ ]:
directory = 'products'
fname = 'top_level_features.csv'
target = f'oci://{bucket_name}@{namespace}/{directory}/{fname}'
target

In [ ]:
X.to_csv(target, index=True, encoding='utf-8',storage_options = config)

In [ ]:
directory = 'products'
fname = hits_file['dec']
target = f'oci://{bucket_name}@{namespace}/{directory}/{fname}'
target

In [ ]:
df_hits_oct = pd.read_csv(target, storage_options={"config": "~/.apikey/config"},low_memory=False)
df_hits_oct.shape

In [ ]:
df_hits_nov = pd.read_csv(target, storage_options={"config": "~/.apikey/config"},low_memory=False)
df_hits_nov.shape

In [ ]:
df_hits_dec = pd.read_csv(target, storage_options={"config": "~/.apikey/config"},low_memory=False)
df_hits_dec.shape

In [ ]:
df_hits_oct['fullVisitorId'] = df_hits_oct['fullVisitorId'].astype(str)
df_hits_nov['fullVisitorId'] = df_hits_nov['fullVisitorId'].astype(str) 
df_hits_dec['fullVisitorId'] = df_hits_dec['fullVisitorId'].astype(str) 

In [ ]:
df_list = [df_hits_oct, df_hits_nov, df_hits_dec]
dfh = pd.concat(df_list, ignore_index=True)
dfh.shape

In [ ]:
dfh['fullVisitorId'].head(5)

In [ ]:
numeric_cols = ['numOfProducts', 'hits.eCommerceAction.action_type', 'hits.time']

In [ ]:
df_hits = init_hits_df(dfh, numeric_cols, 'hits.time')
df_hits.shape

In [ ]:
tmp = df_hits.groupby('fullVisitorId')['fullVisitorId'].agg('count')
len(tmp)

In [ ]:
Y = emotional_rational_hits_features(df_hits)
Y.shape

In [ ]:
Y.isnull().values.any()

In [ ]:
directory = 'features'
fname = 'hits_features_2.csv'
target = f'oci://{bucket_name}@{namespace}/{directory}/{fname}'
target

In [ ]:
Y.to_csv(target, index=True, encoding='utf-8',storage_options = config)

In [ ]:
Z = pd.concat([X,Y], axis=1)
Z.shape

In [ ]:
#Z.drop(columns=['hits.shopping_cart', 'hits.payment_page'], inplace=True)

In [ ]:
fname = 'full_features.csv'
directory = 'features'
target = f'oci://{bucket_name}@{namespace}/{directory}/{fname}'
target

In [ ]:
Z.to_csv(target, index=True, encoding='utf-8',storage_options = config)

In [ ]:
emotional_high = ['trafficSourceRatio', 
                  'trafficSourceSocialRatio', 
                  'mediumSourceFbRatio',
                  'mediumSourceCpcRatio',
                  'hits.products_per_session',
                  'avg_page_views',
                  'hits.avg_viewd_product',
                  'hits.product_page']
emotional_low = ['trafficSourceSeRatio',
                 'mediumSourceOrganicRatio', 
                 'avg_hits',  
                 'avg_time_on_site', 
                 'end_of_month_ratio', 
                 'hits.avg_time2hit', 
                 'hits.avg_time_add_prod',
                 'hits.avg_time_rmv_prod',
                 'hits.support_page',
                 'hits.article_page', 
                 'hits.search_keyword',
                 'hits.category_page']

In [ ]:
def convert_by_z(df, col_list) :
    def f(x, is_max):
        m = np.mean(x)
        s = np.std(x)
        h = m + s
        l =  m - s
        r = h if is_max else l
        return r
    v =  df.apply(lambda y: f(y, True) if y.name in col_list else f(y, False))
    return v 

In [ ]:
Z_lbl = convert_by_z(Z, emotional_high)

In [ ]:
Z_lbl

In [ ]:
directory = 'features'
fname = 'corr_features_vector.csv'
target = f'oci://{bucket_name}@{namespace}/{directory}/{fname}'
target

In [ ]:
def corr_vec_to_cols(df, vec):
    def f(n):
        if n >= 0.05:  # high corr to emotional = 1
            return 'emotio'
        elif n <= -0.05: # low corr to emotional , rational = 2
            return 'ratio'
        else:  # no corr at all
            return 'nocorr'
    dft = df.T
    z = dft.apply(lambda x : x.corr(vec))
    res = z.map(lambda x : f(x))
    return res

In [ ]:
C = corr_vec_to_cols(Z, Z_lbl).to_frame('corr')

In [ ]:
C.head(10)

In [ ]:
len(C.loc[C['corr'] == 'nocorr'])

In [ ]:
len(C.loc[C['corr'] == 'emotio'])

In [ ]:
len(C.loc[C['corr'] == 'ratio'])

In [ ]:
C1.to_csv(target, index=True, encoding='utf-8',storage_options = config)